#  Modelo de ZeeHB

Sample of notebook for specific model

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands
import hep as hp
from matplotlib.colors import LogNorm

In [3]:
def func(s2phi, M1, M2, MAo, Mho, MHo, v, Mu22, Muh2, lam6, lam7):
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo; v2 = v*v
    k = s2phi*np.log(M22/M12)/(4.*np.pi)**2
    phi = 0.5*np.arcsin(s2phi)
    Mu  = (M22-M12)*s2phi/(np.sqrt(2.0)*v)
    MH2 = M12*(np.sin(phi))**2+M22*(np.cos(phi))**2
    M332 = M12*(np.cos(phi))**2+M22*(np.sin(phi))**2
    lam1 = 0.5*(MHo2+Mho2-np.sqrt((MHo2-Mho2)**2-4.0*v2*v2*lam6**2))/(v2)
    lam3 = 2.0*(MH2-Mu22)/(v2)
    lam8 = 2.0*(M332-Muh2)/(v2)
    lam4 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v2*v2*lam6**2)+MHo2+Mho2+2.0*(MAo2-2.0*MH2))/(v2)
    lam5 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v2*v2*lam6**2)+MHo2+Mho2-2.0*MAo2)/(v2)    
    return k, Mu, lam1, lam3, lam4, lam5, lam8 

In [7]:
import hep as hp
def Kappa(s2phi,M1,M2):
    M12 = M1*M1; M22 = M2*M2
    k = s2phi*np.log(M22/M12)/(4.*np.pi)**2
    return k

def Inverse_Zee_Matrix(t12,t13,t23,ml,IH=False,sgnm2=-1,sgnml=1):
    """Inverse neutrino mass matrix normalized by kappa from
        Mnu_diag=U^T.(kappa Y).U
        ,
        (Inverse_Zee_Matrix) = U. Mnu_diag.U^T/kappa
        sgnm2=-1 guarantees real Yukawa couplings
        
    requires hep.py:
       https://github.com/restrepo/BSM-Toolbox/blob/master/tests/hep.py
    """
    if not IH:
        mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(CL=3,IH=False,mnu1in=1E-5*1E-9)
        theta12[1] = t12
        theta13[1] = t13
        theta23[1] = t23
        U=hp.UPMNS(theta12[1],theta13[1],theta23[1])
        m1=ml; m2=sgnm2*np.sqrt(Dm21_2[1]+m1**2); m3=sgnml*np.sqrt(Dm3l_2[1]+m1**2)
    else:
        mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(CL=3,IH=True,mnu1in=1E-5*1E-9)
        #DEBUG signs
        m3=ml; m2=sgnm2*np.sqrt(-Dm3l_2[1]+m3**2); m1=sgnml*np.sqrt(m2**2-Dm21_2[1])
        theta12[1] = t12
        theta13[1] = t13
        theta23[1] = t23
        U=hp.UPMNS(theta12[1],theta13[1],theta23[1])
    
    return np.dot( np.dot( U,np.diag([m1,m2,m3]) ), U.transpose() )

def get_yukawas(X,q00=1.0e-05,q01=1.0e-04,q11=1.0e-03,q20=5.0e-04,q21=1.0e-04,q22=1.0e-06,k=7.18938581886e-05):
    """
    Get the O and f full Yukawa mass matrices which define the neutrino mass matrix in the Zee model
      Mnu=kappa*(O.M_lep.f^T+f.M_lep.O^T)
    where M_lep is the diagonal matrix with the charged leptons
    
    The input X is the output of the `def Inverse_Zee_Matrix(...)` in this module
    """
    import numpy.lib.scimath as sc # .sqrt -> returns complex for sqrt(negative real)
    m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
    ml=np.array([m_e,m_mu,m_tau])
    Q=np.zeros((3,3))
    f=np.zeros((3,3))
    
    Q[0,0]=q00; Q[0,1]=q01; Q[1,1]=q11; Q[2,0]=q20; Q[2,1]=q21; Q[2,2]=q22
    
    Q[0,2] = (ml[2]**2*Q[2,2]*X[0,2]*X[2,2] + ml[2]*(2.0*X[0,2]*(ml[0]*Q[2,0]*X[0,2] + ml[1]*Q[2,1]*X[1,2]) - (ml[0]*(Q[2,0]*X[0,0] + Q[0,0]*X[0,2]) + ml[1]*(Q[2,1]*X[0,1] + Q[0,1]*X[1,2]))*X[2,2]) + np.sqrt(ml[2]**2*((-2.0*X[0,2]*(ml[0]*Q[2,0]*X[0,2] + ml[1]*Q[2,1]*X[1,2]) + (ml[0]*Q[2,0]*X[0,0] + (ml[0]*Q[0,0] - ml[2]*Q[2,2])*X[0,2] + ml[1]*(Q[2,1]*X[0,1] + Q[0,1]*X[1,2]))*X[2,2])**2 - X[2,2]**2*(ml[2]**2*Q[2,2]**2*X[0,0]*X[2,2] + ml[0]**2*X[0,0]*(Q[2,0]**2*X[0,0] - 2.0*Q[0,0]*Q[2,0]*X[0,2] + Q[0,0]**2*X[2,2]) + ml[1]**2*X[1,1]*(Q[2,1]**2*X[0,0] - 2.0*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2]) + 2.0*ml[1]*ml[2]*Q[2,2]*(Q[2,1]*X[0,0]*X[1,2] - Q[0,1]*X[0,1]*X[2,2]) + 2.0*ml[0]*(ml[2]*Q[2,2]*X[0,0]*(Q[2,0]*X[0,2] - Q[0,0]*X[2,2]) + ml[1]*(Q[2,0]*(X[0,1]*(Q[2,1]*X[0,0] - 2.0*Q[0,1]*X[0,2]) + Q[0,1]*X[0,0]*X[1,2]) + Q[0,0]*(-(Q[2,1]*X[0,0]*X[1,2]) + Q[0,1]*X[0,1]*X[2,2])))))))/(ml[2]**2*X[2,2]**2)
    f[0,1] = (ml[0]*(Q[0,1]*Q[2,0] - Q[0,0]*Q[2,1])*X[0,0] + ml[2]*(Q[2,1]*(Q[2,2]*X[0,0] - 2*Q[0,2]*X[0,2]) + Q[0,1]*Q[0,2]*X[2,2]))/(2.*k*ml[1]*(ml[0]*Q[0,1]*(Q[0,1]*Q[2,0] - Q[0,0]*Q[2,1]) + ml[2]*Q[2,1]*(-(Q[0,2]*Q[2,1]) + Q[0,1]*Q[2,2])))
    f[0,2] =-(Q[2,1]**2*X[0,0] - 2*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2])/(2.*k*(ml[0]*Q[0,1]*(Q[0,1]*Q[2,0] - Q[0,0]*Q[2,1]) + ml[2]*Q[2,1]*(-(Q[0,2]*Q[2,1]) + Q[0,1]*Q[2,2])))
    f[1,2] = (ml[2]*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*X[2,2] + ml[0]*(Q[2,0]*(Q[2,1]*X[0,0] - 2*Q[0,1]*X[0,2]) + Q[0,0]*Q[0,1]*X[2,2]))/(2.*k*ml[1]*(ml[0]*Q[0,1]*(Q[0,1]*Q[2,0] - Q[0,0]*Q[2,1]) + ml[2]*Q[2,1]*(-(Q[0,2]*Q[2,1]) + Q[0,1]*Q[2,2])))
    Q[1,0] =-((ml[0]**3*Q[0,0]*(-(Q[0,1]*Q[2,0]) + Q[0,0]*Q[2,1])*X[0,0]*(Q[2,0]*(Q[2,1]*X[0,0] - 2*Q[0,1]*X[0,2]) + Q[0,0]*Q[0,1]*X[2,2]) + ml[2]*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*(ml[2]**2*Q[0,2]*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*X[2,2]**2 + ml[1]**2*Q[2,1]*X[1,1]*(Q[2,1]**2*X[0,0] - 2*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2]) + ml[1]*ml[2]*X[2,2]*(2*Q[2,1]*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*X[0,1] + Q[1,1]*(Q[2,1]*(Q[2,2]*X[0,0] - 2*Q[0,2]*X[0,2]) + Q[0,1]*Q[0,2]*X[2,2]))) + ml[0]**2*(ml[1]*(Q[0,1]*Q[2,0] - Q[0,0]*Q[2,1])*(-(Q[1,1]*X[0,0]) + 2*Q[0,1]*X[0,1])*(Q[2,0]*(-(Q[2,1]*X[0,0]) + 2*Q[0,1]*X[0,2]) - Q[0,0]*Q[0,1]*X[2,2]) + ml[2]*(Q[0,0]*Q[2,2]*X[0,0]*(-2*Q[0,0]*Q[0,1]*Q[2,1]*X[2,2] + Q[2,0]*(-(Q[2,1]**2*X[0,0]) + 2*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2])) + Q[0,2]*(Q[2,0]**2*(Q[2,1]*X[0,0] - 2*Q[0,1]*X[0,2])**2 + Q[0,0]**2*Q[2,1]*(Q[2,1]*X[0,0] + 2*Q[0,1]*X[0,2])*X[2,2] - 2*Q[0,0]*Q[2,0]*X[0,2]*(-(Q[2,1]**2*X[0,0]) + 2*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2])))) - ml[0]*(ml[1]**2*Q[0,1]*(Q[0,1]*Q[2,0] - Q[0,0]*Q[2,1])*X[1,1]*(Q[2,1]**2*X[0,0] - 2*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2]) - ml[2]**2*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*X[2,2]*(-(Q[0,0]*Q[2,1]*Q[2,2]*X[0,0]) + Q[0,2]*(2*Q[2,0]*(Q[2,1]*X[0,0] - 2*Q[0,1]*X[0,2]) + Q[0,0]*(2*Q[2,1]*X[0,2] + Q[0,1]*X[2,2]))) + ml[1]*ml[2]*(-2*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*X[0,1]*(2*Q[0,0]*Q[0,1]*Q[2,1]*X[2,2] + Q[2,0]*(Q[2,1]**2*X[0,0] - 2*Q[0,1]*Q[2,1]*X[0,2] - Q[0,1]**2*X[2,2])) + Q[1,1]*(Q[0,0]*X[2,2]*(-2*Q[0,1]*Q[2,1]*Q[2,2]*X[0,0] + Q[0,2]*(Q[2,1]**2*X[0,0] + 2*Q[0,1]*Q[2,1]*X[0,2] - Q[0,1]**2*X[2,2])) + Q[2,0]*(Q[2,1]**2*X[0,0]*(-(Q[2,2]*X[0,0]) + 2*Q[0,2]*X[0,2]) + Q[0,1]**2*(Q[2,2]*X[0,0] + 2*Q[0,2]*X[0,2])*X[2,2] + 2*Q[0,1]*Q[2,1]*(Q[2,2]*X[0,0]*X[0,2] - Q[0,2]*(2*X[0,2]**2 + X[0,0]*X[2,2])))))))/(ml[0]*ml[1]*(Q[2,1]**2*X[0,0] - 2*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2])*(ml[0]*(-(Q[0,1]*Q[2,0]) + Q[0,0]*Q[2,1])*X[0,0] - ml[2]*(Q[2,1]*(Q[2,2]*X[0,0] - 2*Q[0,2]*X[0,2]) + Q[0,1]*Q[0,2]*X[2,2]))))
    Q[1,2] = (ml[0]**2*Q[0,0]*(-(Q[0,1]*Q[2,0]) + Q[0,0]*Q[2,1])*X[0,0] + ml[0]*(-(ml[1]*(Q[0,1]*Q[2,0] - Q[0,0]*Q[2,1])*(-(Q[1,1]*X[0,0]) + 2.0*Q[0,1]*X[0,1])) + ml[2]*(-(Q[0,0]*Q[2,1]*Q[2,2]*X[0,0]) + Q[0,2]*(2.0*Q[0,0]*Q[2,1]*X[0,2] + Q[2,0]*(Q[2,1]*X[0,0] - 2.0*Q[0,1]*X[0,2])))) + ml[2]*(ml[2]*Q[0,2]*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*X[2,2] + ml[1]*(2.0*Q[2,1]*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])*X[0,1] + Q[1,1]*(Q[2,1]*(Q[2,2]*X[0,0] - 2.0*Q[0,2]*X[0,2]) + Q[0,1]*Q[0,2]*X[2,2]))))/(ml[1]*ml[2]*(Q[2,1]**2*X[0,0] - 2.0*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2]))

    
    f[1,0]=-f[0,1]; f[2,0]=-f[0,2]; f[2,1]=-f[1,2]
    
    return Q,f

def test_Zee(q00,q01,q11,q20,q21,q22):
    m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
    mlep=np.diag([m_e,m_mu,m_tau])
    m1=0 # lightest neutrino 
    s2phi=1.4E-02; M1=200.0; M2=300.0
    t12 = 33.56*np.pi/180.0 
    t13 = 8.46*np.pi/180.0 
    t23 = 41.6*np.pi/180.0
    X=Inverse_Zee_Matrix(t12,t13,t23,m1)
    k=Kappa(s2phi,M1,M2)
    O,f=get_yukawas(X,q00,q01,q11,q20,q21,q22,k) #Use default input values
    #print O
    #print f
    Mnu=k*( np.dot( np.dot(O,mlep),f.transpose() )+np.dot( np.dot(f,mlep),O.transpose()) )
    #print Mnu
    Mnu_diag,U=np.linalg.eig(Mnu)
    lo=np.argsort(np.abs(Mnu_diag))
    Mnu_diag=np.array([Mnu_diag[lo[0]],Mnu_diag[lo[1]],Mnu_diag[lo[2]]])
    U=np.matrix(U)
    U=np.asarray(np.hstack((U[:,lo[0]],U[:,lo[1]],U[:,lo[2]])))
    
    mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(CL=3,IH=False,mnu1in=1E-5*1E-9)
    m2=np.sqrt(Dm21_2[1]+m1**2); m3=np.sqrt(Dm3l_2[1]+m1**2)
    theta12[1] = t12
    theta13[1] = t13
    theta23[1] = t23
    print Mnu_diag
    print np.abs([m1,m2,m3])
    np.testing.assert_array_almost_equal(np.abs(U),\
        np.abs( hp.UPMNS(theta12[1],theta13[1],theta23[1]) ) )
    
    return np.testing.assert_almost_equal( np.abs(Mnu_diag),np.abs([m1,m2,m3]) ) 

In [8]:
test_Zee(q00=1e-02,q01=1e-3,q11=1e-2,q20=5e-1,q21=-1e-3,q22=1e-3)

[-2.33127925e-25 -8.66025404e-12  5.02394267e-11]
[0.00000000e+00 8.66025404e-12 5.02394267e-11]



## Check one point

In [9]:
a=hp.hep(MODEL='radinuZeeHB')

`a-object` is an object with many attributes and methods. Use the tab to explore them. Some of them are
* a.Series: [pandas](http://pandas.pydata.org/) Series object with the "relevant" variables 
* a.LHA: Input LesHouces file as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runSPheno() -> a.LHA_out: return LHA output files as [pyslha](https://pypi.python.org/pypi/pyslha/) object
* a.runmicromegas() -> a.runSPheno() -> Updated the `a-object`  with micrOMEGAS "relevant" output

In [10]:
pd.Series(a.LHA.blocks['MINPAR'].entries)

1      1.0000000E-01   # lambda1Input
2      1.3000000E-01   # lambda2Input
3      1.1000000E+00   # lambda3Input
4      -5.0000000E-01  # lambda4Input
5      5.0000000E-01   # lambda5Input
6      0.000000E+00    # lambda6Input
7      0.000000E+00    # lambda7Input
8      0.000000E+00    # lambda8Input
9      0.000000E+00    # lambda9Input
10    0.000000E+00    # lambda10Input
11     0.000000E+00    # lambdahInput
12          0.000000E+00    # MhInput
13          0.000000E+00    # MuInput
14        4.0000000E+04   # mEt2Input
dtype: object

for i in something....

In [11]:
v = a.vev
s2phi=1.4E-02 
M1=350.0 
M2=M1-50.0
Mho = 125.0
MHo = M1+10.0
MAo = MHo+5.0
Mu22 = 200.0**2
Muh2 = 300.0**2

lam6 = 1.000000E-03     # lambda6Inputd 
lam7 = 1.000000E-02     # lambda7Input

lam2 = 1.3000000E-01    # lambda2Input
lam9 = 0.000000E+00     # lambda9Input
lam10 = 0.000000E+00    # lambda10Input
lamh = 0.000000E+00     # lambdahInput


k, Mu, lam1, lam3, lam4, lam5, lam8 = func(s2phi, M1, M2, MAo, Mho, MHo, v, Mu22, Muh2, lam6, lam7)
print k, Mu, lam1, lam3, lam4, lam5, lam8

-2.733277634389549e-05 -1.3066884974335997 0.25773432452768236 1.6495488117759323 1.3661422169590498 -0.05979477179826951 1.0721200382534934


In [15]:
Me   = 0.5109989461e-3     #Masa Electron en GeV
Mmu  = 0.1056583745        #Masa Muon en GeV
Mtau = 1.77686             #Masa Tauon en GeV

df = pd.DataFrame()

devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]='%0.8E       #lambda1Input' %lam1
a.LHA.blocks['MINPAR'][2]='%0.8E       #lambda2Input' %lam2
a.LHA.blocks['MINPAR'][3]='%0.8E       #lambda3Input' %lam3
a.LHA.blocks['MINPAR'][4]='%0.8E       #lambda4Input' %lam4
a.LHA.blocks['MINPAR'][5]='%0.8E       #lambda5Input' %lam5
a.LHA.blocks['MINPAR'][6]='%0.8E       #lambda6Input' %lam6
a.LHA.blocks['MINPAR'][7]='%0.8E       #lambda7Input' %lam7
a.LHA.blocks['MINPAR'][8]='%0.8E       #lambda8Input' %lam8
a.LHA.blocks['MINPAR'][9]='%0.8E       #lambda9Input' %lam9
a.LHA.blocks['MINPAR'][10]='%0.8E      #lambda10Input'%lam10
a.LHA.blocks['MINPAR'][11]='%0.8E      #lambdahInput' %lamh
a.LHA.blocks['MINPAR'][12]='%0.8E      #MhInput'      %Muh2
a.LHA.blocks['MINPAR'][13]='%0.8E      #MuInput'      %Mu
a.LHA.blocks['MINPAR'][14]='%0.8E      #mEt2Input'    %Mu22

m3=1.0e-17  # lightest neutrino
t12=33.56*180.0/np.pi
t13=8.49*180.0/np.pi
t23=41.6*180.0/np.pi
X=Inverse_Zee_Matrix(t12,t13,t23,m3,IH=True)
O,f=get_yukawas(X,q00=-4.255172e-03,q01=2.412983e-05,q11=3.109212e-11,q20=-4.987462e-04,q21=-2.227795e-05,q22=3.246787e-08)
    
Yh11 = 0.0        # Yh(1,1)
Yh12 = f[0,1]     # Yh(1,2)
Yh13 = f[0,2]     # Yh(1,3)
Yh21 = f[1,0]     # Yh(2,1)
Yh22 = 0.0        # Yh(2,2)
Yh23 = f[1,2]     # Yh(2,3)
Yh31 = f[2,0]     # Yh(3,1)
Yh32 = f[2,1]     # Yh(3,2)
Yh33 = 0.0        # Yh(3,3)

a.LHA.blocks['YHIN'][(1,1)]='%0.8E      # Yh(1,1)'    %Yh11
a.LHA.blocks['YHIN'][(1,2)]='%0.8E      # Yh(1,2)'    %Yh12
a.LHA.blocks['YHIN'][(1,3)]='%0.8E      # Yh(1,3)'    %Yh13
a.LHA.blocks['YHIN'][(2,1)]='%0.8E      # Yh(2,1)'    %Yh21
a.LHA.blocks['YHIN'][(2,2)]='%0.8E      # Yh(2,2)'    %Yh22
a.LHA.blocks['YHIN'][(2,3)]='%0.8E      # Yh(2,3)'    %Yh23
a.LHA.blocks['YHIN'][(3,1)]='%0.8E      # Yh(3,1)'    %Yh31
a.LHA.blocks['YHIN'][(3,2)]='%0.8E      # Yh(3,2)'    %Yh32
a.LHA.blocks['YHIN'][(3,3)]='%0.8E      # Yh(3,3)'    %Yh33

epsE11 = O[0,0]          # epsE(1,1)
epsE12 = O[0,1]          # epsE(1,2)
epsE13 = O[0,2]          # epsE(1,3)
epsE21 = O[1,0]          # epsE(2,1)
epsE22 = O[1,1]          # epsE(2,2)
epsE23 = O[1,2]          # epsE(2,3)
epsE31 = O[2,0]          # epsE(3,1)
epsE32 = O[2,1]          # epsE(3,2)
epsE33 = O[2,2]          # epsE(3,3)

a.LHA.blocks['EPSEIN'][(1,1)]='%0.8E      # epsE(1,1)'    %epsE11
a.LHA.blocks['EPSEIN'][(1,2)]='%0.8E      # epsE(1,2)'    %epsE12
a.LHA.blocks['EPSEIN'][(1,3)]='%0.8E      # epsE(1,3)'    %epsE13
a.LHA.blocks['EPSEIN'][(2,1)]='%0.8E      # epsE(2,1)'    %epsE21
a.LHA.blocks['EPSEIN'][(2,2)]='%0.8E      # epsE(2,2)'    %epsE22
a.LHA.blocks['EPSEIN'][(2,3)]='%0.8E      # epsE(2,3)'    %epsE23
a.LHA.blocks['EPSEIN'][(3,1)]='%0.8E      # epsE(3,1)'    %epsE31
a.LHA.blocks['EPSEIN'][(3,2)]='%0.8E      # epsE(3,2)'    %epsE32
a.LHA.blocks['EPSEIN'][(3,3)]='%0.8E      # epsE(3,3)'    %epsE33
    
tanbetha = 1.0

Yd33 = 0.0240086062
Yu33 = 0.996529469
epsD33 =-Yd33*tanbetha
epsU33 = Yu33*tanbetha 
    
a.LHA.blocks['EPSDIN'][(3,3)]='%0.8E      # epsD(3,3)'    %epsD33
a.LHA.blocks['EPSUIN'][(3,3)]='%0.8E      # epsU(3,3)'    %epsU33

moc=a.runSPheno() 

a.Series=a.Series.append(hp.block_to_series(a.LHA_out_with_comments.blocks['MASS']))
a.branchings(a.LHA_out.decays,min_pdg=25)
a.Series=a.Series.append(a.Br_names)
a.Series=a.Series.append(hp.block_to_series(a.LHA_out_with_comments.blocks['FLAVORKITLFV']))

df=df.append(a.Series,ignore_index=True)
df=df.fillna(0)

In [16]:
m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
mlep=np.diag([m_e,m_mu,m_tau])
Mnu = k*( np.dot( np.dot(O,mlep),f.transpose() )+np.dot( np.dot(f,mlep),O.transpose()) )
Mnu_diag,U=np.linalg.eig(Mnu)
lo=np.argsort(np.abs(Mnu_diag))
Mnu_diag=np.array([Mnu_diag[lo[0]],Mnu_diag[lo[1]],Mnu_diag[lo[2]]])
U=np.matrix(U)
U=np.asarray(np.hstack((U[:,lo[0]],U[:,lo[1]],U[:,lo[2]])))
print Mnu_diag # m
print U


[-3.80182356e-18 -1.87757743e-11  1.90622690e-11]
[[-0.48441403 -0.85882061  0.16664393]
 [ 0.7188595  -0.28219721  0.63529974]
 [-0.49858206  0.42754168  0.75407164]]


In [17]:
a.LHA_out_with_comments.blocks['MASS'].entries

_dict([(25, '1.25000000E+02  # hh_1'),
       (35, '3.60000000E+02  # hh_2'),
       (37, '3.00000000E+02  # Hm_2'),
       (900037, '3.50000000E+02  # Hm_3'),
       (36, '3.65000000E+02  # etI'),
       (23, '9.11887000E+01  # VZ'),
       (24, '8.03497269E+01  # VWp'),
       (1, '5.00000000E-03  # Fd_1'),
       (3, '9.50000000E-02  # Fd_2'),
       (5, '4.18000000E+00  # Fd_3'),
       (2, '2.50000000E-03  # Fu_1'),
       (4, '1.27000000E+00  # Fu_2'),
       (6, '1.73500000E+02  # Fu_3'),
       (11, '5.10998930E-04  # Fe_1'),
       (13, '1.05658372E-01  # Fe_2'),
       (15, '1.77669000E+00  # Fe_3'),
       (12, '3.03523722E-16  # Fv_1'),
       (14, '-1.91763920E-11 # Fv_2'),
       (16, '1.94780148E-11  # Fv_3')])

[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./) 
[Jupyter home](./draft.pdf)